## Tutorial: Synchrotron and Synchrotron Self Compton
In this tutorial we will show how to compute the Spectral Energy Distribution produced by the Synchrotron and Synchrotron Self-Compton radiative processes in a jetted AGN.

In [ ]:
# import numpy, astropy and matplotlib for basic functionalities
import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.coordinates import Distance
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
# import agnpy classes
from agnpy.emission_regions import Blob
from agnpy.synchrotron import Synchrotron
from agnpy.compton import SynchrotronSelfCompton
from agnpy.utils.plot import plot_sed, load_mpl_rc
load_mpl_rc()

### Emission Region
Let us consider that the emission region is a spherical plasmoid of radius $R_b = 10^{16}\,\mathrm{cm}$, streaming with Lorentz factor $\Gamma=10$ in the AGN jet, resulting in a Doppler boosting of $\delta_D=10$ (the jet is almost aligned with the observer's line of sight). The galaxy hosting the active nucleus is at a distance of $d_L=10^{27}\,\mathrm{cm}$. The blob has a tangled magnetic field $B=1\,\mathrm{G}$. The electron spectrum is described by a power law with index $-2.8$ and a total energy content of $W_e = 10^{48}\,{\rm erg}$.

In [ ]:
# total energy content of the electron distribution
spectrum_norm = 1e48 * u.Unit("erg") 
# dictionary describing the electron distribution
spectrum_dict = {
    "type": "PowerLaw", 
    "parameters": {
        "p": 2.8, 
        "gamma_min": 1e2, 
        "gamma_max": 1e7
    }
}
R_b = 1e16 * u.cm
B = 1 * u.G
z = Distance(1e27, unit=u.cm).z
delta_D = 10
Gamma = 10
# emission region
blob = Blob(R_b, z, delta_D, Gamma, B, spectrum_norm, spectrum_dict)

In [ ]:
# the blob object is printable, returning a summary of its properties
print(blob)

In [ ]:
# we can also print the total electrons number and energy
print(f"total particle number: {blob.N_e_tot:.2e}")
print(f"total energy in electrons: {blob.W_e:.2e}")

In [ ]:
# as well as the jet power in particles and magnetic fields (see the documentation for more details)
print(f"jet power in particles: {blob.P_jet_e:.2e}")
print(f"jet power in magnetic field: {blob.P_jet_B:.2e}")

### Synchrotron Radiation
To intitialise the object that will compute the synchrotron radiation, we simply pass the `Blob` instance to the `Synchrotron` class. The synchrotron self absorption can be considered when computing the SED.

In [ ]:
synch = Synchrotron(blob)
synch_ssa = Synchrotron(blob, ssa=True)

In [ ]:
# let us define now a grid of frequencies over which to calculate the synchrotron SED
nu_syn = np.logspace(8, 23) * u.Hz
# let us compute a synchrotron, and a self-absorbed synchrotron SED
synch_sed = synch.sed_flux(nu_syn)
synch_sed_ssa = synch_ssa.sed_flux(nu_syn)

In [ ]:
plot_sed(nu_syn, synch_sed, color="k", label="synchr.")
plot_sed(nu_syn, synch_sed_ssa, ls="--", color="gray", label="self absorbed synchr.")
plt.show()

### Synchrotron Self-Compton Radiation
Similarly to the synchrotron case, to intitialise the object that will compute the synchrotron self-Compton (SSC) radiation, we simply pass the `Blob` instance to the `SynchrotronSelfCompton` class.       
We examine also the case produced by scattering over a self-absorbed synchrotron spectrum.

In [ ]:
# simple ssc
ssc = SynchrotronSelfCompton(blob)
# ssc over a self-absorbed synchrotron spectrum
ssc_ssa = SynchrotronSelfCompton(blob, ssa=True)

In [ ]:
nu_ssc = np.logspace(15, 30) * u.Hz
sed_ssc = ssc.sed_flux(nu_ssc)
sed_ssc_ssa = ssc_ssa.sed_flux(nu_ssc)

In [ ]:
plot_sed(nu_ssc, sed_ssc, color="k", label="SSC")
plot_sed(nu_ssc, sed_ssc_ssa, ls="--", color="gray", label="SSC with SSA")
plt.show()

### Literature Check from Dermer and Menon (2009)
In order to provide a cross-check with the literature, we reproduce Figure 7.4 of Dermer and Menon (2009) - [see bibliography in the docs](https://agnpy.readthedocs.io/en/latest/bibliography.html). The figure, reported here, depicts two complete synchrotron and SSC SEDs, generated by the same emission regions distinguished only by the different maximum Lorentz factor of their electron distributions ($\gamma_{\rm max} = 10^5$ and $\gamma_{\rm max} = 10^7$). 

In [ ]:
Image("figures/figure_7_4_dermer_2009.png", width = 600, height = 400)

The `Blob` previosuly defined is already set with the parameters reproducing the example with $\gamma_{\rm max} = 10^7$, let us create a new `Blob` containing an electron distribution with $\gamma_{\rm max} = 10^5$.

In [ ]:
spectrum_dict["parameters"]["gamma_max"] = 1e5
blob2 = Blob(R_b, z, delta_D, Gamma, B, spectrum_norm, spectrum_dict)
synch2 = Synchrotron(blob2)
ssc2 = SynchrotronSelfCompton(blob2)

In [ ]:
plot_sed(nu_syn, synch.sed_flux(nu_syn), color="k", label=r"${\rm synch},\,\gamma_{\rm max}=10^7$")
plot_sed(nu_ssc, ssc.sed_flux(nu_ssc), color="k", label=r"${\rm SSC},\,\gamma_{\rm max}=10^7$")
plot_sed(nu_syn, synch2.sed_flux(nu_syn), color="crimson", label=r"${\rm synch},\,\gamma_{\rm max}=10^5$")
plot_sed(nu_ssc, ssc2.sed_flux(nu_ssc), color="crimson", label=r"${\rm SSC},\,\gamma_{\rm max}=10^5$")
# select the same x and y range of the figure
plt.xlim([1e9, 1e30])
plt.ylim([1e-12, 1e-9])
plt.show()